In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

                                              0.0/991.5 kB ? eta -:--:--
     -                                       30.7/991.5 kB 1.4 MB/s eta 0:00:01
     ----                                   112.6/991.5 kB 1.7 MB/s eta 0:00:01
     ---------                              235.5/991.5 kB 1.8 MB/s eta 0:00:01
     ------------------                     491.5/991.5 kB 2.8 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 4.5 MB/s eta 0:00:01
     -------------------------------------- 991.5/991.5 kB 4.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch


In [2]:
import torch
print(torch.__version__)


2.2.0+cu118


In [3]:
tokenzier = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Atul kishore\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [13]:
token = tokenzier.encode('I hate surfing in summers', return_tensors='pt')

In [14]:
token[0]

tensor([  101,   151, 39487, 60529, 10285, 10104, 58096,   102])

In [8]:
tokenzier.decode(token[0])

'[CLS] i love surfing in summers [SEP]'

In [15]:
res = model(token)

In [16]:
res

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.6516,  0.1220, -0.5550, -0.4740,  0.1520]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
torch.argmax(res.logits)

tensor(0)

In [18]:
int(torch.argmax(res.logits))+1

1

In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("raw_reviews_train.csv")

In [22]:
df.head()

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",Negative
1,"The saddest thing about this ""tribute"" is that...",Negative
2,Last night I decided to watch the prequel or s...,Negative
3,I have to admit that i liked the first half of...,Negative
4,I was not impressed about this film especially...,Negative


In [26]:
df = df.drop(columns='sentiment')

In [27]:
df.head()

,text
0,"Now, I won't deny that when I purchased this o..."
1,"The saddest thing about this ""tribute"" is that..."
2,Last night I decided to watch the prequel or s...
3,I have to admit that i liked the first half of...
4,I was not impressed about this film especially...


In [29]:
df['text'].iloc[0]

"Now, I won't deny that when I purchased this off eBay, I had high expectations. This was an incredible out-of-print work from the master of comedy that I so enjoy. However, I was soon to be disappointed. Apologies to those who enjoyed it, but I just found the Compleat Al to be very difficult to watch. I got a few smiles, sure, but the majority of the funny came from the music videos (which I've got on DVD) and the rest was basically filler. You could tell that this was not Al's greatest video achievement (that honor goes to UHF). Honestly, I doubt if this will ever make the jump to DVD, so if you're an ultra-hardcore Al fan and just HAVE to own everything, buy the tape off eBay. Just don't pay too much for it."

In [30]:
def sentiment_score(text):
    tokens = token = tokenzier.encode(text, return_tensors='pt')
    res = model(token)
    return int(torch.argmax(res.logits))+1

In [32]:
sentiment_score(df['text'].iloc[6])

3

In [33]:
df["sentiment"] = df['text'].apply(lambda x: sentiment_score(x[:512]))

In [34]:
df.head()

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",2
1,"The saddest thing about this ""tribute"" is that...",1
2,Last night I decided to watch the prequel or s...,1
3,I have to admit that i liked the first half of...,2
4,I was not impressed about this film especially...,2


In [35]:
df.shape

(7499, 2)

In [36]:
df.tail()

,text,sentiment
7494,I'm amazed we see even one nay-sayer criticizi...,4
7495,Absolutely one of the worst movies I have ever...,1
7496,Arthur is middle aged rich 'kid' who drinks li...,4
7497,A River Runs Through It is based on the true s...,4
7498,Holy crap this is so hysterical! Why aren't Am...,1


In [38]:
# Save the model
model.save_pretrained('./sentiment_model')

# Save the tokenizer
tokenzier.save_pretrained('./sentiment_model')


('./sentiment_model\\tokenizer_config.json',
 './sentiment_model\\special_tokens_map.json',
 './sentiment_model\\vocab.txt',
 './sentiment_model\\added_tokens.json',
 './sentiment_model\\tokenizer.json')